In [13]:
#| default_exp datasets

# Data Loading

## SBS Narrative Recall Dataset

In [14]:
#|export
import numpy as np
import json
from torch.utils.data import Dataset
import os

class SBS_NarrativeDataset(Dataset):
    def __init__(self, data_directory):
        
        # each response file represents a sample; these are named based on '{story_name}_{subject_id}_{iteration}'
        self.data_directory = data_directory
        self.text_directory = os.path.join(data_directory, 'texts')
        self.sequence_directory = os.path.join(data_directory, 'sequences', 'human')
        self.response_files = []
        self.stories = {}
        for path, subdirs, files in os.walk(self.text_directory):
            for name in files:
                if name.count('_') == 2:
                    self.response_files.append(name)
                else:
                    with open(os.path.join(path, name), 'r', encoding='utf-8') as f:
                        self.stories[name[:-4]] = f.read()

        self.trial_indices = np.arange(len(self.response_files))

    def _retrieve_story_text(self, response_file_name):
        story_name = response_file_name.split('_')[0]
        return self.stories[story_name]

    def _retrieve_response_text(self, response_file_name):
        story_name = response_file_name.split('_')[0]
        with open(os.path.join(self.text_directory, story_name, response_file_name), 'r') as f:
            return f.read()
        
    def _retrieve_response_sequence(self, response_file_name):
        story_name = response_file_name.split('_')[0]
        with open(os.path.join(self.sequence_directory, story_name, response_file_name[:-3]+'json'), 'r') as f:
            return json.load(f)
        
    def _prepare_match_matrix(self, response_sequence):
        matchings = response_sequence['correspondences']
        match_matrix = np.zeros(
            (len(response_sequence['source_units']), len(response_sequence['response_units'])), dtype=bool)

        for response_index, matched_target in enumerate(matchings):
            if matched_target > -1:
                match_matrix[matched_target, response_index] = True
        return match_matrix
    
    def _prepare_target_items(self, story_text, response_sequence):
        updated_unit_start = [story_text.find(unit) for unit in response_sequence['source_units']]
        
        for index, unit in enumerate(response_sequence['source_units']):
            assert(updated_unit_start[index] > -1)

        return [{'text': unit, 'spans':[(start, start+len(unit))]} for unit, start in zip(
            response_sequence['source_units'], updated_unit_start)]
    
    def _prepare_response_units(self, response_text, response_sequence):
        updated_text = [
            response_text[span[0]:span[1]].strip() for span in response_sequence['response_spans']]

        return [{'text': unit, 'spans':[(span[0], span[0]+len(unit))]} for unit, span in zip(
            updated_text, response_sequence['response_spans'])]

    def __len__(self):
        return len(self.response_files)
    
    def __getitem__(self, idx):
        
        trials = []
        for trial_idx in np.atleast_1d(self.trial_indices[idx]):

            response_file_name = self.response_files[trial_idx]
            story_text = self._retrieve_story_text(response_file_name)
            response_text = self._retrieve_response_text(response_file_name)
            response_sequence = self._retrieve_response_sequence(response_file_name)

            trials.append({
                'target_context': story_text, 
                'target_items': self._prepare_target_items(story_text, response_sequence), 
                'response_transcript': response_text, 
                'response_units': self._prepare_response_units(response_text, response_sequence), 
                'matches': self._prepare_match_matrix(response_sequence)})
            
        return trials[0] if len(trials) == 1 else trials

In [15]:
data_directory = 'C:/Users/gunnj/compmempy/data/narrative'
dataset = SBS_NarrativeDataset(data_directory)

example_entry = dataset[0]
example_entry

{'target_context': 'It was a hot, sunny day and Kaylie and Rachel decided to take advantage of their off day and take a trip to the beach. The sun was an optimistic yellow pellet, blistering in the sky as they were on their way to Old Orchard Beach in Maine. Once arriving and walking from the parking lot, they spotted the crystal clear water that faded into a deep blue on the horizon. The beach was swarming with people. Some on yellow, orange, white, and pink towels, while others were seen stretched out on lounge chairs with broad-brimmed hats shading their eyes from the rays of the sun. Children were decorating sandcastles with smooth, round seashells, and Kaylie decided to stretch out on her towel in the sand for a quick nap while Rachel went to cool off in the ocean. \nThe sun shined on the water, causing it to look like a million little crystals. Just before making it to the surf, Rachel saw a little girl with freckles in a light purple bathing suit and little yellow floaties on he

## Senses Dataset

In [16]:
#| export
import numpy as np
from torch.utils.data import Dataset
import chardet
import hdf5storage

class SensesDataset(Dataset):
    def __init__(self, hdf5_file_path, sense_pool_path, transform=None):
        """
        Initialize the dataset.

        Args:
            hdf5_file_path (str): Path to the HDF5 file.
            sense_pool_path (str): Path to the sense pool file.
            transform (callable, optional): Optional transform to be applied
                on a sample. Defaults to None.
        """
        # Load the data from the specified HDF5 file
        # You can customize this part to load your data
        self.data = hdf5storage.read(path='/data', filename=hdf5_file_path)

        self.trial_count = len(self.data["subject"])
        self.trial_indices = np.arange(self.trial_count)
        with open(sense_pool_path, mode='rb') as f:
            raw_data = f.read()
            detected_encoding = chardet.detect(raw_data)['encoding']
            self.sense_pool = raw_data.decode(detected_encoding).split('\n')
        
        # Set the transform if provided
        self.transform = transform

    def __len__(self):
        """
        Return the total number of samples in the dataset.

        Returns:
            int: Number of trials in the dataset.
        """
        return self.trial_count

    def __getitem__(self, idx):
        """
        Get a trial from the dataset at the specified index.

        Args:
            trial_idx (int): Index of the trial to retrieve.

        Returns:
            sample: The retrieved sample.
        """

        # pres_itemids selects the indices from sense_pool of the target items
        trials = []
        for trial_idx in np.atleast_1d(self.trial_indices[idx]):
            senses = self.data['pres_itemids'][trial_idx]
            target_items = [self.sense_pool[each-1].strip() for each in senses if each != 0]
            
            # response units are the segments of the transcript selected by raters and their spans in the transcript
            response_transcript = str(self.data['recall_transcript'][trial_idx][0])
            response_units = [str(each) for each in self.data['response_units'][trial_idx] if str(each) != '']

            # full response units include text *and* span representations contained in a dict
            response_start_spans = [each-1 for each in self.data["response_unit_start"][trial_idx] if each != 0]
            response_end_spans = [each-1 for each in self.data["response_unit_end"][trial_idx] if each != 0]
            full_response_units = [{'text': unit, 'spans':[(start_span, end_span)]} for unit, start_span, end_span in zip(
                response_units, response_start_spans, response_end_spans)]
            
            # matchings
            match_matrix = np.zeros((len(target_items), len(response_units)), dtype=bool)
            target_indices = np.array([each-1 for each in self.data['recalls'][trial_idx] if each != 0])

            if len(target_indices) > 0:
                match_matrix[target_indices, np.arange(len(response_units))] = True

            trials.append({
                'target_context': '', 
                'target_items': target_items, 
                'response_transcript': response_transcript, 
                'response_units': full_response_units, 
                'matches': match_matrix})
            
        return trials[0] if len(trials) == 1 else trials

In [17]:
section_tag = 'base' # unique identifier for this variation of notebook parameters
output_dir = 'C:/Users/gunnj/workspace/response_sequencer/data/'

dataset = SensesDataset(os.path.join(output_dir, f'{section_tag}_senses.h5'), os.path.join(output_dir, f'{section_tag}_sense_pool.txt'))

dataset[:2]

[{'target_context': '',
  'target_items': ['(n) ace (one of four playing cards in a deck having a single pip on its face)',
   '(n) ace, adept, champion, sensation, maven, mavin, virtuoso, genius, hotshot, star, superstar, whiz, whizz, wizard, wiz (someone who is dazzlingly skilled in any field)',
   '(n) ace (a serve that the receiver is unable to reach)',
   '(v) breeze through, ace, pass with flying colors, sweep through, sail through, nail (succeed at easily) "She sailed through her exams"; "You will pass with flying colors"; "She nailed her astrophysics course"',
   '(v) ace (serve an ace against (someone))',
   '(n) one, 1, I, ace, single, unity (the smallest whole number or a numeral representing this number) "he has the one but will need a two and three to go with it"; "they had lunch at one"',
   '(adj) ace, A-one, crack, first-rate, super, tiptop, topnotch, top-notch, tops (of the highest quality) "an ace reporter"; "a crack shot"; "a first-rate golfer"; "a super party"; "pla

In [18]:
#| hide
import nbdev; nbdev.nbdev_export()